## **Etude pratique: le chemin des deneigeuses**

### **1 - Les import**
Dans ce fichier, nous utilisons 2 librairies. Deux sont utilisées pour le parcours du drone. **`osnmx`** pour générer le graph à partir d'une ville ou d'un quartier et **`networkX`** pour le traitement du graphe. 

In [19]:
import osmnx as ox
import networkx as nx

### **2 - Fonction d'appel**
La fonction `loadingGraph` prend en paramètre _`neighborhood`_ le quartier, _`city`_ la ville, _`country`_ le pays et permet de charger le graphe.
Pour ce qui est des paramètres completion et maps il s'agit de flag permettant d'afficher respectivement le niveau de
couverture du graphe par les deneigeuses et les trajets que vont parcourir les deneigeuses.
Nous aborderons l'intérêt des trois derniers paramètres dans la fonction `find_critical`.

In [15]:
def main(neighborhood,city,country,required_len_pull,required_len, min_accuracy, completion, maps):
    if (neighborhood):
        G = ox.graph_from_place(neighborhood + ", " + city + ", " + country, network_type='drive')
    else:
        G = ox.graph_from_place(city + ", " + country, network_type='drive')
    G = nx.convert_node_labels_to_integers(G)
    list_ = []
    path = dict(nx.all_pairs_shortest_path(G))
    all_path = create_path_pull(path, required_len)
    res_path = find_critical_road(G, path, all_path, min_accuracy, required_len, completion, maps)

### **3 - Création des chemins**
La fonction `create_path_pull` prend en paramètre _`path`_ le dictionnaire contenant tout les shortest path du graph et _`required_len`_ qui permet de selectionner par la longueur les chemins que nous allons attribuer à nos deneigeuses plus tard dans la suite du code.

In [ ]:
def create_path_pull(path, required_len):
    all_path = []
    for elem in path.values():
        for var in elem.values():
            if len(var) > required_len:
                all_path.append(var)
    return all_path

### **4 - Détermination de la rentabilité**
Il s'agit de la première sous fonction utilisée dans la fonction `find_critical_road`. L'objectif de cette fonction est de determiné la pertinence d'un chemin par rapport aux sommets que nous avons déjà visité. L'objectif étant de minimiser le nombre de fois ou nous allons repasser par des sommets que nous avons visité tout en maximisant la longueur de nos vecteurs.

In [21]:
def rentability(checked_mat, need_to_happend):
    res = 0
    for elem in need_to_happend:
        if checked_mat[elem] != 1:
            res+=1
    return res/len(need_to_happend)

### **5 - Mise à jour du vecteur de parcours**
Tout comme précedemment, il s'agit d'une sous fonction utilisée dans la fonction `find_critical_road`. L'objectif de la fonction est de mettre a jour la liste des sommets par rapport au chemin que nous venons d'ajouter dans a liste de nos chemins finaux. Si jamais un nouveau sommet est rencontré on passe la valeur contenu à son index a True.

In [ ]:
def update(checked_mat, res_path):
    for elem in res_path:
        if checked_mat[elem] != True:
            checked_mat[elem] = True
    return checked_mat

In [ ]:
def find_critical_road(G, path, all_path, min_accuracy, required_len, completion, maps):
    checked_mat = [False] * len(path.values())
    critical_floor = min_accuracy
    acceptable_len = required_len
    critical_road = []
    length = len(all_path) - 1
    i = 0
    while False in checked_mat:
        if i > length:
            i = 0
            critical_floor -= 0.05
            acceptable_len -= 1
            if completion:
                print("The complete cover is at :", (checked_mat.count(True)*100/len(checked_mat)), "% of its completion")
        if len(all_path[i]) > acceptable_len:
            if rentability(checked_mat, all_path[i]) > critical_floor:
                critical_road.append(all_path[i])
                if maps:
                    fig, ax = ox.plot_graph_route(G, all_path[i], route_linewidth=4, node_size=1, bgcolor='k')
                checked_mat = update(checked_mat, all_path[i])
        i+=1
    return critical_road

In [18]:
main(None,"Gentilly", "France", 4, 15, 0.9, True, False)

The complete cover is at : 37.93103448275862 % of its completion
The complete cover is at : 45.97701149425287 % of its completion
The complete cover is at : 45.97701149425287 % of its completion
The complete cover is at : 52.87356321839081 % of its completion
The complete cover is at : 69.54022988505747 % of its completion
The complete cover is at : 69.54022988505747 % of its completion
The complete cover is at : 69.54022988505747 % of its completion
The complete cover is at : 69.54022988505747 % of its completion
The complete cover is at : 74.13793103448276 % of its completion
The complete cover is at : 74.13793103448276 % of its completion
The complete cover is at : 74.13793103448276 % of its completion
The complete cover is at : 74.13793103448276 % of its completion
The complete cover is at : 80.45977011494253 % of its completion
The complete cover is at : 82.75862068965517 % of its completion
The complete cover is at : 82.75862068965517 % of its completion
The complete cover is at 